In [14]:
import pandas as pd
import json
import io
import requests

from docxtpl import DocxTemplate
from docxcompose.composer import Composer
from docx import Document

from docx.enum.text import WD_BREAK
from sys import getsizeof
from pprint import pprint

In [15]:
try:
    with open('../files/keys.json') as f:
        data = json.load(f)
        gsheet_perigrammata_id = data['gsheet_perigrammata']
        print(gsheet_perigrammata_id)
except:
    print(2)
    pass

1qOLxB2BNYvTLiTxofUSJCUVd7JIdkcflPTcR_FLub5k


In [16]:
def replace_none_with_empty_str(some_dict):
    return {k: ('' if v is None else v) for k, v in some_dict.items()}

## Load Google Sheet

In [17]:
def load_gsheet(lang: str) -> pd.DataFrame:
    sheet_id = gsheet_perigrammata_id
    if lang == "gr":
        sheet_name = "gr_2025"
    if lang == "eng":
        sheet_name = "eng"
    url = fr"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    df = pd.read_csv(url, dtype_backend='pyarrow', index_col=0)
    return df

In [18]:
df = load_gsheet('gr')
df.head()

,code,name,school,department,level,examino,didactivities_name1,didactivities_hours1,didactivities_ects1,didactivities_name2,...,hours3,task4,hours4,task5,hours5,hours_sum,grading,refs_books,refs_more_title,refs_more
id,,,,,,,,,,,,,,,,,,,,,
1,ΓΕΝ001,Απειροστικός Λογισμός Ι,ΜΗΧΑΝΙΚΩΝ,ΠΟΛΙΤΙΚΩΝ ΜΗΧΑΝΙΚΩΝ,Προπτυχιακό,1,Διαλέξεις – Ασκήσεις Πράξης,4,5,None,...,26,<NA>,<NA>,<NA>,<NA>,130,Τελική Αξιολόγηση του Μαθήματος: • Γραπτή ...,"Τερζίδης Χαράλαμπος, Λογισμός συναρτήσεων μιας...",None,None
2,ΓΕΝ002,Γραμμική Άλγεβρα και Αναλυτική Γεωμετρία,ΜΗΧΑΝΙΚΩΝ,ΠΟΛΙΤΙΚΩΝ ΜΗΧΑΝΙΚΩΝ,Προπτυχιακό,1,Διαλέξεις – Ασκήσεις Πράξης,3,5,None,...,26,<NA>,<NA>,<NA>,<NA>,130,Τελική Αξιολόγηση του Μαθήματος: • Γραπτή ...,"Τερζίδης Χαράλαμπος, Λογισμός συναρτήσεων μιας...",None,None
3,ΓΕΝ003,Φυσική για Μηχανικούς,ΜΗΧΑΝΙΚΩΝ,ΠΟΛΙΤΙΚΩΝ ΜΗΧΑΝΙΚΩΝ,Προπτυχιακό,1,"Διαλέξεις, Ασκήσεις Πράξης και Εργαστηριακές Α...",5,5,None,...,6,Συγγραφή Εργασιών,9,Αυτοτελής Μελέτη,82,156,Αξιολόγηση Θεωρίας (90% του τελικού βαθμού): ...,"Young H., Freedman R., Πανεπιστημιακή φυσική μ...",None,None
4,ΓΕΝ009,Βασικά εργαλεία πληροφορικής - Συγγραφής επιστ...,ΜΗΧΑΝΙΚΩΝ,ΠΟΛΙΤΙΚΩΝ ΜΗΧΑΝΙΚΩΝ,Προπτυχιακό,1,Διαλέξεις,1,0,None,...,<NA>,<NA>,<NA>,<NA>,<NA>,20,Γραπτή τελική εξέταση που περιλαμβάνει: • Θ...,"Καβουσανός Ε., Εφαρμογές Μαθηματικού Λογισμού ...",None,None
5,ΔΟΜ001,Τεχνικό Σχέδιο,ΜΗΧΑΝΙΚΩΝ,ΠΟΛΙΤΙΚΩΝ ΜΗΧΑΝΙΚΩΝ,Προπτυχιακό,1,"Διαλέξεις, Ασκήσεις Πράξης και Εργαστηριακές Α...",4,5,None,...,35,Αυτοτελής Μελέτη,<NA>,Ατομική Εργασία,43,125,"α. Τελική γραπτή εξέταση: Ερωτήσεις θεωρίας, ...","Μαλικούτη, Στ., «ΤΕΧΝΙΚΟ ΣΧΕΔΙΟ: Στοιχεία θεωρ...",None,None


## Make Word Files

In [19]:
def make_word_file(row_dict: dict, lang: str):
    if lang == "gr":
        url = r"https://github.com/panagop/civil_ihu_pyappz/raw/main/files/perigrammata-template-gr.docx"
    if lang == "eng":
        url = r"https://github.com/panagop/civil_ihu_pyappz/raw/main/files/perigrammata-template-eng.docx"

    response = requests.get(url, timeout=5)
    bytes_io = io.BytesIO(response.content)

    doc = DocxTemplate(bytes_io)

    doc.render(row_dict)
    buffer = io.BytesIO()
    doc.save(buffer)

    return buffer.getvalue()

In [20]:
def save_word_file(bytes_stream, file_path):
    with open(file_path, "wb") as file:
        file.write(bytes_stream)

In [21]:
filepath = 'gen2'

for index, row in df.iterrows():
    row_dict = row.to_dict()
    row_dict = replace_none_with_empty_str(row_dict)
    
    word_file_buffer = make_word_file(row_dict, 'gr')  # Change 'eng' to 'gr' for Greek files
    
    filename = f'{index:03d}'+'-'+row_dict['code']
    
    save_word_file(bytes_stream = word_file_buffer,
               file_path=filepath+'\\' + filename + '.docx')
    
    

In [12]:
# filepath = 'gen'

# row_dict = df.iloc[2].to_dict()
# row_dict = replace_none_with_empty_str(row_dict)
# word_file_buffer = make_word_file(row_dict, 'eng')

# filename = row_dict['name']+'-'+row_dict['code']
# save_word_file(bytes_stream = word_file_buffer,
#                file_path=filepath+'\\' + filename + '.docx')

In [ ]:
# merged_document = Document('Perigrammata-template-gr.docx')
# composer = Composer(merged_document)
# for path in document_paths:
#     d = Document(path)
#     d.add_page_break()
#     composer.append(d)
# composer.save('merged_document.docx')

In [ ]:
# merged_document = Document('../files/perigrammata-template-eng.docx')
# composer = Composer(merged_document)
# d = Document(r'D:\mypython_projects\civil_ihu_pyappz\jupyter\gen')
# for path in document_paths:
#     d = Document(path)
#     d.add_page_break()
#     composer.append(d)
# composer.save('merged_document.docx')

In [ ]:
# import glob
# from docx import Document

# def merge_multiple_documents(documents):
#     """
#     Merge multiple Document objects into a new Document object and return it.
#     """
#     new_document = Document('../files/perigrammata-template-eng.docx')

#     for doc in documents:
#         for element in doc.element.body:
#             new_document.element.body.append(element)
        
#         new_document.add_page_break()

#     return new_document

# # Get a list of all .docx files in the folder
# file_paths = glob.glob(r'D:\mypython_projects\civil_ihu_pyappz\jupyter\gen/*.docx')

# # Open each file and add it to a list of Document objects
# documents = [Document(path) for path in file_paths]

# # Merge the documents
# merged_document = merge_multiple_documents(documents)

# # Save the merged document
# merged_document.save('merged.docx')

In [ ]:
# with open('../files/perigrammata-template-eng.docx', 'rb') as f:
#     bytes_io = io.BytesIO(f.read())

In [ ]:
# bytes_io

In [ ]:
# doc = DocxTemplate(bytes_io)
# doc.undeclared_template_variables

{'code',
 'department',
 'didactivities_ects1',
 'didactivities_ects2',
 'didactivities_hours1',
 'didactivities_hours2',
 'didactivities_name1',
 'didactivities_name2',
 'erasmus',
 'examino',
 'grading',
 'hours1',
 'hours2',
 'hours3',
 'hours4',
 'hours5',
 'hours_sum',
 'lang',
 'learn_objectives',
 'level',
 'name',
 'prereq',
 'refs_books',
 'school',
 'skills',
 'subject1',
 'subject2',
 'subject3',
 'task1',
 'task2',
 'task3',
 'task4',
 'task5',
 'teaching_way',
 'techs_used',
 'type',
 'website'}

In [17]:
# url = r"https://github.com/panagop/civil_ihu_pyappz/raw/daf2ec082269559dcc28c8675b0a55178fd3b122/civil_ihu_pyappz/perigrammata-template-eng.docx"
# response = requests.get(url)
# bytes_io = io.BytesIO(response.content)

In [16]:
# doc = DocxTemplate(bytes_io)
# doc.undeclared_template_variables

In [23]:
doc = DocxTemplate("perigrammata-template-gr.docx")

doc_examino = DocxTemplate("Εξάμηνο-template-gr.docx")

In [24]:
doc.undeclared_template_variables


{'code',
 'department',
 'didactivities_ects1',
 'didactivities_ects2',
 'didactivities_hours1',
 'didactivities_hours2',
 'didactivities_name1',
 'didactivities_name2',
 'erasmus',
 'examino',
 'grading',
 'hours1',
 'hours2',
 'hours3',
 'hours4',
 'hours5',
 'hours_sum',
 'lang',
 'learn_objectives',
 'level',
 'name',
 'prereq',
 'prereq_knowledge',
 'refs_books',
 'school',
 'skills',
 'subject1',
 'subject2',
 'subject3',
 'task1',
 'task2',
 'task3',
 'task4',
 'task5',
 'teaching_way',
 'techs_used',
 'type',
 'website'}

In [ ]:
# sheet_id = gsheet_perigrammata_id
# sheet_name = "eng"
# url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# # df = pd.read_excel('Περιγράμματα-Δεδομένα (gr-eng).xlsx', sheet_name='gr', dtype_backend='pyarrow')
# # df = df.drop([0])

# df = pd.read_csv(url, dtype_backend='pyarrow')

# # df = df.fillna('')


## Merge Word Files


In [29]:
document_paths = []
for index, row in df.iterrows():
    examino_path = f"gen2\{row['examino']:03d}_examino.docx"
    if examino_path not in document_paths:
        doc_examino.render({'examino': row['examino']})
        doc_examino.save(examino_path)
        document_paths.append(examino_path)
        print(examino_path)
    row_dict = row.to_dict()
    row_dict = replace_none_with_empty_str(row_dict)
    doc.render(row_dict)
    path = f"gen2\{row_dict['examino']:03d}_{row_dict['code']}.docx"
    doc.save(path)
    document_paths.append(path)

<string>:3: SyntaxWarning: invalid escape sequence '\{'
<string>:12: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\{'
<>:12: SyntaxWarning: invalid escape sequence '\{'
<string>:3: SyntaxWarning: invalid escape sequence '\{'
<string>:12: SyntaxWarning: invalid escape sequence '\{'
<>:3: SyntaxWarning: invalid escape sequence '\{'
<>:12: SyntaxWarning: invalid escape sequence '\{'
C:\Users\panagop\AppData\Local\Temp\ipykernel_34240\1572263597.py:3: SyntaxWarning: invalid escape sequence '\{'
  examino_path = f"gen2\{row['examino']:03d}_examino.docx"
C:\Users\panagop\AppData\Local\Temp\ipykernel_34240\1572263597.py:12: SyntaxWarning: invalid escape sequence '\{'
  path = f"gen2\{row_dict['examino']:03d}_{row_dict['code']}.docx"


gen2\001_examino.docx
gen2\002_examino.docx
gen2\003_examino.docx
gen2\004_examino.docx
gen2\005_examino.docx
gen2\006_examino.docx
gen2\007_examino.docx
gen2\008_examino.docx
gen2\009_examino.docx
gen2\010_examino.docx


In [32]:
document_paths[:20]

['gen2\\001_examino.docx',
 'gen2\\001_ΓΕΝ001.docx',
 'gen2\\001_ΓΕΝ002.docx',
 'gen2\\001_ΓΕΝ003.docx',
 'gen2\\001_ΓΕΝ009.docx',
 'gen2\\001_ΔΟΜ001.docx',
 'gen2\\001_ΔΟΜ002.docx',
 'gen2\\001_ΣΥΓ001.docx',
 'gen2\\002_examino.docx',
 'gen2\\002_ΓΕΝ004.docx',
 'gen2\\002_ΓΕΝ005.docx',
 'gen2\\002_ΓΕΝ006.docx',
 'gen2\\002_ΔΟΜ003.docx',
 'gen2\\002_ΔΟΜ005.docx',
 'gen2\\002_ΔΟΜ037.docx',
 'gen2\\002_ΣΥΓ002.docx',
 'gen2\\003_examino.docx',
 'gen2\\003_ΓΕΝ007.docx',
 'gen2\\003_ΓΕΩ001.docx',
 'gen2\\003_ΔΟΜ038.docx']

In [31]:
merged_document = Document('Perigrammata-template-gr.docx')
composer = Composer(merged_document)
for path in document_paths:
    d = Document(path)
    d.add_page_break()
    composer.append(d)
composer.save('merged_document_2025gr.docx')